## First look at Gyro data

Craig Lage 03-Jan-24

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from astropy.time import Time, TimeDelta

from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')

# Times to start looking at data
start = Time("2024-01-02 22:00:00Z", scale='utc')
end = Time("2024-01-02 23:00:00Z", scale='utc')

gyro = await client.select_time_series("lsst.sal.MTM1M3.gyroData", \
                                       ['*'], start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                        ['*'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                        ['*'],  start, end)    

In [ ]:
azV = az['actualVelocity'].values
azT = az['timestamp'].values

elP = el['actualPosition'].values
elV = el['actualVelocity'].values
elT = el['timestamp'].values

# Gyro readings are in radians/sec
gyroX = gyro['angularVelocityX'].values * -180.0 / np.pi
gyroY = gyro['angularVelocityY'].values * -180.0 / np.pi
gyroT = gyro['timestamp'].values

# Need to divide gyroY by cos(elevation)
interpElP = np.interp(gyroT, elT, elP) * np.pi / 180.0
gyroY /= np.cos(interpElP)


In [ ]:
fig, axs = plt.subplots(2,2, figsize=(12,10))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

axs[0][0].plot(gyroT, gyroY, color='red', label="Gyro angularVelocityY")
axs[0][0].plot(azT, azV, ls='--', lw=2, color='black', label="Azimuth velocity")
plotStart = Time("2024-01-02T22:26:20", scale='utc').unix_tai
plotEnd = Time("2024-01-02T22:29:00", scale='utc').unix_tai
axs[0][0].set_title("Azimuth", fontsize=18)
axs[0][0].set_xlim(plotStart, plotEnd)
ax2 = axs[0][0].twinx()
ax2.plot(elT, elP, label='Elevation')
axs[0][0].set_ylim(-5,6)
axs[0][0].set_xlabel("Time(sec)")
axs[0][0].set_ylabel("angular velocity (deg/sec)")
axs[0][0].legend(loc='upper left')
ax2.set_ylabel("Elevation(degrees)")
ax2.legend()

axs[0][1].plot(gyroT, gyroX, color='red', label="Gyro angularVelocityX")
axs[0][1].plot(elT, elV, ls='--', lw=2, color='black', label="Elevation velocity")
plotStart = Time("2024-01-02T22:22:30", scale='utc').unix_tai
plotEnd = Time("2024-01-02T22:23:30", scale='utc').unix_tai
axs[0][1].set_title("Elevation", fontsize=18)
axs[0][1].set_xlim(plotStart, plotEnd)
axs[0][1].set_ylim(-2.5,2.5)
axs[0][1].set_xlabel("Time(sec)")
axs[0][1].set_ylabel("angular velocity (deg/sec)")
axs[0][1].legend(loc='upper left')

axs[1][0].plot(gyroT, gyroY, color='red', label="Gyro angularVelocityY", marker='x')
axs[1][0].plot(azT, azV, ls='--', lw=2, color='black', label="Azimuth velocity", marker='o')
plotStart = Time("2024-01-02T22:28:18.4", scale='utc').unix_tai
plotEnd = Time("2024-01-02T22:28:18.8", scale='utc').unix_tai
axs[1][0].set_title("Azimuth", fontsize=18)
axs[1][0].set_xlim(plotStart, plotEnd)
axs[1][0].set_ylim(-0.2,0.8)
axs[1][0].set_xlabel("Time(sec)")
axs[1][0].set_ylabel("angular velocity (deg/sec)")
axs[1][0].legend(loc='upper left')

axs[1][1].plot(gyroT, gyroX, color='red', label="Gyro angularVelocityX", marker='x')
axs[1][1].plot(elT, elV, ls='--', lw=2, color='black', label="Elevation velocity", marker='o')
plotStart = Time("2024-01-02T22:23:11.8", scale='utc').unix_tai
plotEnd = Time("2024-01-02T22:23:12.8", scale='utc').unix_tai
axs[1][1].set_title("Elevation", fontsize=18)
axs[1][1].set_xlim(plotStart, plotEnd)
axs[1][1].set_ylim(-0.2, 0.8)
axs[1][1].set_xlabel("Time(sec)")
axs[1][1].set_ylabel("angular velocity (deg/sec)")
axs[1][1].legend(loc='upper left')


plt.savefig('/home/c/cslage/u/MTM1M3/data/Gyro_Angular_Velocity_03Jn24.png')